In [16]:
import os
import pandas as pd
import numpy as np
import re
import string
import json
import zipfile
import matplotlib.pyplot as plt
from os.path import isfile, join
from random import sample 
import ast
from json.decoder import JSONDecodeError
import seaborn as sns
from functools import reduce
import warnings
import tqdm
from tqdm.notebook import tqdm, trange
import time
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm, trange

In [36]:
import os
import pandas as pd
import numpy as np
import re
import string
import json
import zipfile
import matplotlib.pyplot as plt
from os.path import isfile, join
from random import sample 
import ast
from json.decoder import JSONDecodeError
import seaborn as sns
from functools import reduce
import warnings
import tqdm
from tqdm.notebook import tqdm, trange
import time
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm, trange

# files = [file for file in os.listdir(directory) if file.endswith('.json')]

tweet_file_nested = [json.loads(line) for line in open('data/0905_162649.json', 'r')]
tweet_file = tweet_file_nested[0]

# add user and place info to each tweet
for i in range(len(tweet_file['data'])):
    for j in range(len(tweet_file['includes']['users'])):
        try:

            if tweet_file['data'][i]['author_id'] == tweet_file['includes']['users'][j]['id']:
                #ここでuserinfoを設定
                tweet_file['data'][i]['user_info'] = tweet_file['includes']['users'][j] 

            if 'in_reply_to_user_id' in tweet_file['data'][i]:
                    if tweet_file['data'][i]['in_reply_to_user_id'] == tweet_file['includes']['users'][j]['id']:
                        tweet_file['data'][i]['in_reply_to_user_info'] = tweet_file['includes']['users'][j]

#             for k in range(len(tweet_file['includes']['places'])):

#                 if 'geo' in tweet_file['data'][i]:
#                     if 'place_id' in tweet_file['data'][i]['geo']:
#                         if tweet_file['data'][i]['geo']['place_id'] == tweet_file['includes']['places'][k]['id']:
#                             tweet_file['data'][i]['place_info'] = tweet_file['includes']['places'][k]

#             if 'referenced_tweets.id.author_id' in tweet_file['data'][i]:
#               if tweet_file['data'][i]['referenced_tweets.id.author_id'] == tweet_file['includes']['users'][j]['id']:
#                   tweet_file['data'][i]['referenced_user_info'] = tweet_file['includes']['users'][j]

        except KeyError:
            pass

        if "referenced_tweets" in tweet_file['data'][i]:
            if "tweets" in tweet_file['includes']:
                for k in range(len(tweet_file['includes']['tweets'])):
                    if tweet_file['data'][i]['referenced_tweets'][0]['id'] == tweet_file['includes']['tweets'][k]['id']:
                        tweet_file['data'][i]['referenced_tweets_info'] = tweet_file['includes']['tweets'][k]
        
#         if "referenced_tweets" in tweet_file['data'][i]:
#             if tweet_file['data'][i]['referenced_tweets_info']['author_id'] == tweet_file['includes']['users'][j]['id']:
#                  tweet_file['data'][i]['referenced_user_info'] = tweet_file['includes']['users'][j]


print("Done with user and place objects...")


tweets = [tweet_file['data'][i] for i in range(len(tweet_file['data']))]
print("Loaded a sample of", len(tweet_file), "files and", len(tweets), "tweets")

Done with user and place objects...
Loaded a sample of 3 files and 19 tweets


## Data Frame作成

In [34]:
%%time
## info
tweet_id = []
created_at = []
conversation_id = []
text = []

# ## place
# place_full_name = []
# place_country_code = []
# place_country = []
# place_type = []
# place_bbox = []
# place_from_text = []

## Public metrics
retweet_count = []
reply_count = []
like_count = []
quote_count = []

# user info
user_id = []
username = []
user_full_name = []
user_following = []
user_followers = []
# user_location = []
user_created_at = []
user_bio = []

# in reply to user info
in_reply_to_user_id = []
in_reply_to_username = []
in_reply_to_user_full_name = []
in_reply_to_user_following = []
in_reply_to_user_followers = []
# in_reply_to_user_location = []
# in_reply_to_user_created_at = []
in_reply_to_user_bio = []

# # referenced user info
# referenced_user_id = []
# referenced_username = []
# referenced_user_full_name = []
# referenced_user_following = []
# referenced_user_followers = []
# referenced_user_bio = []

# referenced tweets info
referenced_tweet_id = []
referenced_tweet_type = []
referenced_tweet_text = []
referenced_tweet_created_at = []
referenced_tweet_author_id = []
referenced_tweet_conversation_id = []

# # hexagon_id info
# hex_ids = []

for tweet in tqdm(tweets, desc = "data preprocessing"):
    
    '''tweet info'''
    # tweet id
    tweet_id.append(tweet['id'])
    # tweet text
    text.append(tweet['text'])
    # created_at
    created_at.append(tweet['created_at'])
    # conversation_id
    conversation_id.append(tweet['conversation_id'])
    # retweet_count
    retweet_count.append(tweet['public_metrics']['retweet_count'])
    # retweet_count
    quote_count.append(tweet['public_metrics']['quote_count'])
    # retweet_count
    like_count.append(tweet['public_metrics']['like_count'])
    # retweet_count
    reply_count.append(tweet['public_metrics']['reply_count'])
#     # hashtags
#     tags = []
#     try:
#         for hashtag in tweet['entities']['hashtags']:
#             tags.append(hashtag['tag'])
#         hashtags.append(tags)
#     except KeyError:
#         hashtags.append(np.NaN)
    
#     '''tweet place info'''
#    
    
    '''user info'''
    if 'user_info' in tweet.keys():
        # get user id
        user_id.append(tweet['user_info']['id'])
        # get user username
        username.append(tweet['user_info']['username'])
        # get user user_full_name
        user_full_name.append(tweet['user_info']['name'])
        # get user friends
        user_following.append(tweet['user_info']['public_metrics']['following_count'])
        # get user followers
        user_followers.append(tweet['user_info']['public_metrics']['followers_count'])
        # get user bio
        user_bio.append(tweet['user_info']['description'])
    else:
        user_id.append(np.NaN)
        username.append(np.NaN)
        user_full_name.append(np.NaN)
        user_following.append(np.NaN)
        user_followers.append(np.NaN)
        user_bio.append(np.NaN)
        
    '''in reply to user info'''
    if 'in_reply_to_user_info' in tweet.keys():
        # get user id
        in_reply_to_user_id.append(tweet['in_reply_to_user_info']['id'])
        # get user username
        in_reply_to_username.append(tweet['in_reply_to_user_info']['username'])
        # get user user_full_name
        in_reply_to_user_full_name.append(tweet['in_reply_to_user_info']['name'])
        # get user friends
        in_reply_to_user_following.append(tweet['in_reply_to_user_info']['public_metrics']['following_count'])
        # get user followers
        in_reply_to_user_followers.append(tweet['in_reply_to_user_info']['public_metrics']['followers_count'])
#         # get user_created_at
#         in_reply_to_user_created_at.append(tweet['in_reply_to_user_info']['created_at'])
        # get user bio
        in_reply_to_user_bio.append(tweet['in_reply_to_user_info']['description'])
    else:
        in_reply_to_user_id.append(np.NaN)
        in_reply_to_username.append(np.NaN)
        in_reply_to_user_full_name.append(np.NaN)
        in_reply_to_user_following.append(np.NaN)
        in_reply_to_user_followers.append(np.NaN)
#         in_reply_to_user_created_at.append(np.NaN)
        in_reply_to_user_bio.append(np.NaN)
        

#      '''referenced user info'''
#     if 'in_reply_to_user_info' in tweet.keys():
#         # get user id
#         in_reply_to_user_id.append(tweet['in_reply_to_user_info']['id'])
#         # get user username
#         in_reply_to_username.append(tweet['in_reply_to_user_info']['username'])
#         # get user user_full_name
#         in_reply_to_user_full_name.append(tweet['in_reply_to_user_info']['name'])
#         # get user friends
#         in_reply_to_user_following.append(tweet['in_reply_to_user_info']['public_metrics']['following_count'])
#         # get user followers
#         in_reply_to_user_followers.append(tweet['in_reply_to_user_info']['public_metrics']['followers_count'])
# #         # get user_created_at
# #         in_reply_to_user_created_at.append(tweet['in_reply_to_user_info']['created_at'])
#         # get user bio
#         in_reply_to_user_bio.append(tweet['in_reply_to_user_info']['description'])
#     else:
#         in_reply_to_user_id.append(np.NaN)
#         in_reply_to_username.append(np.NaN)
#         in_reply_to_user_full_name.append(np.NaN)
#         in_reply_to_user_following.append(np.NaN)
#         in_reply_to_user_followers.append(np.NaN)
# #         in_reply_to_user_created_at.append(np.NaN)
#         in_reply_to_user_bio.append(np.NaN)
    
#       '''referenced user info'''
    
    if 'referenced_tweets_info' in tweet.keys():
        # get tweet id
        referenced_tweet_id.append(tweet['referenced_tweets'][0]['id'])
        # get tweet type
        referenced_tweet_type.append(tweet['referenced_tweets'][0]['type'])
        # get tweet text
        referenced_tweet_text.append(tweet['referenced_tweets_info']['text'])
        # get tweet created_at
        referenced_tweet_created_at.append(tweet['referenced_tweets_info']['created_at'])
        # get tweet author id
        referenced_tweet_author_id.append(tweet['referenced_tweets_info']['author_id'])
        # get tweet conversation id
        referenced_tweet_conversation_id.append(tweet['referenced_tweets_info']['conversation_id'])
    else:
        referenced_tweet_id.append(np.NaN)
        referenced_tweet_type.append(np.NaN)
        referenced_tweet_text.append(np.NaN)
        referenced_tweet_created_at.append(np.NaN)
        referenced_tweet_author_id.append(np.NaN)
        referenced_tweet_conversation_id.append(np.NaN)

#     '''hexagon collection info'''
#     hex_ids.append(tweet['hex_id'])
    
# df columns
cols = ['tweet_id', 'conversation_id', 'created_at', 'text', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'user_id', 
        'username', 'user_full_name', 'user_following', 'user_followers', 'user_bio', 
        'in_reply_to_user_id', 'in_reply_to_username', 'in_reply_to_user_full_name', 'in_reply_to_user_following', 
        'in_reply_to_user_followers', 'in_reply_to_user_bio', 
        'referenced_tweet_id', 'referenced_tweet_type', 'referenced_tweet_text', 'referenced_tweet_created_at', 
        'referenced_tweet_author_id', 'referenced_tweet_conversation_id']

tweets_df = pd.DataFrame(list(zip(tweet_id,conversation_id,created_at,text,retweet_count, 
                                  reply_count,like_count,quote_count,user_id,username,user_full_name,user_following, 
                                  user_followers,user_bio,in_reply_to_user_id,
                                  in_reply_to_username,in_reply_to_user_full_name, in_reply_to_user_following,
                                  in_reply_to_user_followers, in_reply_to_user_bio, referenced_tweet_id, referenced_tweet_type, referenced_tweet_text, 
                                  referenced_tweet_created_at, referenced_tweet_author_id, referenced_tweet_conversation_id
                                  )), columns = cols)

print("Processed", len(tweets_df), "tweets")

tweets_df = tweets_df.drop_duplicates(subset="tweet_id")
tweets_df

data preprocessing:   0%|          | 0/19 [00:00<?, ?it/s]

Processed 19 tweets
CPU times: user 25.3 ms, sys: 6.17 ms, total: 31.5 ms
Wall time: 33.7 ms


,tweet_id,conversation_id,created_at,text,retweet_count,reply_count,like_count,quote_count,user_id,username,...,in_reply_to_user_full_name,in_reply_to_user_following,in_reply_to_user_followers,in_reply_to_user_bio,referenced_tweet_id,referenced_tweet_type,referenced_tweet_text,referenced_tweet_created_at,referenced_tweet_author_id,referenced_tweet_conversation_id
0,1555338306900045824,1555338306900045824,2022-08-04T23:42:03.000Z,RT @tokyonewsroom: 安倍元首相の国葬 抗議行動や反対声明相次ぐ\n\n米モ...,277,0,0,0,174574802,akahata_PR,...,NaN,NaN,NaN,NaN,1554983831090139136,retweeted,安倍元首相の国葬 抗議行動や反対声明相次ぐ\n\n米モンタナ州立大准教授の山口智美さんは「旧...,2022-08-04T00:13:30.000Z,810015588198338560,1554983831090139136
1,1555143272863252482,1555143272863252482,2022-08-04T10:47:04.000Z,RT @sitosiyoko8: @yukioiokibe 統一協会の教義＝自民党の憲法草案...,4,0,0,0,448585293,murai_akemi,...,NaN,NaN,NaN,NaN,1554657590331981825,retweeted,"@yukioiokibe 統一協会の教義＝自民党の憲法草案\n日本は""エバ国家""で「サタン（...",2022-08-03T02:37:08.000Z,873194570,1554392837013131265
2,1555000429356122113,1555000429356122113,2022-08-04T01:19:27.000Z,RT @hotaka_tsukada: 山谷先生、一年生議員の時から、統一教会系『世界日報』...,1743,0,0,0,5780912,HYamaguchi,...,NaN,NaN,NaN,NaN,1554837312789368832,retweeted,山谷先生、一年生議員の時から、統一教会系『世界日報』（2001.11.25・26）で、「結婚...,2022-08-03T14:31:17.000Z,3118622494,1554829553650257920
3,1554989297597554688,1554989297597554688,2022-08-04T00:35:13.000Z,RT @tokyonewsroom: 安倍元首相の国葬 抗議行動や反対声明相次ぐ\n\n米モ...,277,0,0,0,268213306,tokyo_shimbun,...,NaN,NaN,NaN,NaN,1554983831090139136,retweeted,安倍元首相の国葬 抗議行動や反対声明相次ぐ\n\n米モンタナ州立大准教授の山口智美さんは「旧...,2022-08-04T00:13:30.000Z,810015588198338560,1554983831090139136
4,1554983831090139136,1554983831090139136,2022-08-04T00:13:30.000Z,安倍元首相の国葬 抗議行動や反対声明相次ぐ\n\n米モンタナ州立大准教授の山口智美さんは「旧...,277,18,474,15,810015588198338560,tokyonewsroom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1554949146825543680,1554949146825543680,2022-08-03T21:55:40.000Z,RT @hotaka_tsukada: 山谷先生、一年生議員の時から、統一教会系『世界日報』...,1743,0,0,0,95014181,TomoMachi,...,NaN,NaN,NaN,NaN,1554837312789368832,retweeted,山谷先生、一年生議員の時から、統一教会系『世界日報』（2001.11.25・26）で、「結婚...,2022-08-03T14:31:17.000Z,3118622494,1554829553650257920
6,1554946699122331648,1554946699122331648,2022-08-03T21:45:57.000Z,RT @hotaka_tsukada: 山谷先生、一年生議員の時から、統一教会系『世界日報』...,1743,0,0,0,113548325,levinassien,...,NaN,NaN,NaN,NaN,1554837312789368832,retweeted,山谷先生、一年生議員の時から、統一教会系『世界日報』（2001.11.25・26）で、「結婚...,2022-08-03T14:31:17.000Z,3118622494,1554829553650257920
7,1554791581605298179,1554791581605298179,2022-08-03T11:29:34.000Z,RT @yamtom: 話題の井上義行議員、この動画でも家庭教育を推し、別姓に反対しているな...,54,0,0,0,268582226,R_KitanoR,...,NaN,NaN,NaN,NaN,1554762196554375168,retweeted,話題の井上義行議員、この動画でも家庭教育を推し、別姓に反対しているな。\n安倍総理の元首席秘...,2022-08-03T09:32:48.000Z,52646196,1554762196554375168
8,1554779651972939776,1554779651972939776,2022-08-03T10:42:10.000Z,山谷氏は選択的夫婦別姓について「ファミリーネームの廃止になるから反対だ」と主張している。ht...,85,2,164,2,116641739,KazuhiroSoda,...,NaN,NaN,NaN,NaN,1554669278166536192,quoted,「私は関係ありません」旧統一教会の支援否定 自民・山谷えり子氏\nhttps://t.co/...,2022-08-03T03:23:34.000Z,49540955,1554669278166536192
9,1554732978181328896,1554732978181328896,2022-08-03T07:36:42.000Z,RT @nana77rey1: 必聴回。\n山上容疑者がTwitterでジェンダーについて（...,9,0,0,0,4171231,tsuda,...,NaN,NaN,NaN,NaN,1554724467590324224,retweeted,必聴回。\n山上容疑者がTwitterでジェンダーについて（インセルの立場で）も相当数言及。...,2022-08-03T07:02:53.000Z,949521025863069697,1554724467590324224


In [8]:
tweets_df.to_csv("tweets_1.csv")